In [48]:
# Quintuplet Brides
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

%matplotlib inline
np.random.seed(198)

In [49]:
# reset all variables/placeholders
tf.reset_default_graph()

In [ ]:
# flatten samples to (1, 64*64*3).T


In [50]:
def create_placeholders(n_x, n_y):
    
    X = tf.placeholder(shape=[n_x, None], dtype=tf.float32)
    Y = tf.placeholder(shape=[n_y, None], dtype=tf.float32)
    
    return X, Y

In [51]:
print(str(create_placeholders(12288, 5)))

(<tf.Tensor 'Placeholder:0' shape=(12288, ?) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(5, ?) dtype=float32>)


In [87]:
 def initialize_parameters():
        """
        network - [num_of_sample, 20, 10, 6]
        """
        W1 = tf.cast(tf.get_variable("W1", shape=[20, 12288], initializer=tf.keras.initializers.he_normal(seed=1)), tf.float32)
        b1 = tf.cast(tf.get_variable("b1", shape=[20, 1], initializer=tf.zeros_initializer(), dtype=tf.float32), tf.float32)
        W2 = tf.cast(tf.get_variable("W2", shape=[10, 20],  initializer=tf.keras.initializers.he_normal(seed=1)), tf.float32)
        b2 = tf.cast(tf.get_variable("b2", shape=[10, 1], initializer=tf.zeros_initializer(), dtype=tf.float32), tf.float32)
        W3 = tf.cast(tf.get_variable("W3", shape=[5, 10], initializer=tf.keras.initializers.he_normal(seed=1)), tf.float32)
        b3 = tf.cast(tf.get_variable("b3", shape=[5, 1], initializer=tf.zeros_initializer()), tf.float32)
        
        parameters = {
            "W1": W1,
            "b1": b1,
            "W2": W2,
            "b2": b2,
            "W3": W3,
            "b3": b3
        }
        return parameters

In [93]:
tf.reset_default_graph()
initialize_parameters()

{'W1': <tf.Tensor 'W1/read:0' shape=(20, 12288) dtype=float32>,
 'b1': <tf.Tensor 'b1/read:0' shape=(20, 1) dtype=float32>,
 'W2': <tf.Tensor 'W2/read:0' shape=(10, 20) dtype=float32>,
 'b2': <tf.Tensor 'b2/read:0' shape=(10, 1) dtype=float32>,
 'W3': <tf.Tensor 'W3/read:0' shape=(5, 10) dtype=float32>,
 'b3': <tf.Tensor 'b3/read:0' shape=(5, 1) dtype=float32>}

In [99]:
def forward_propagation(X, parameters):
    # retrieve
    W1, b1, W2, b2, W3, b3 = [i for i in parameters.values()]

    # forwad propagation
    Z1 = tf.add(tf.matmul(W1, X), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)
    
    return Z3

In [100]:
tf.reset_default_graph()

with tf.Session() as sess:
    X, Y = create_placeholders(12288, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    print("Z3:" + str(Z3))

Z3:Tensor("Add_2:0", shape=(5, ?), dtype=float32)


In [ ]:
compute_cost(Z3, Y)

In [ ]:
# Thanks to Andrew
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.0001,
          num_epochs = 1500, minibatch_size = 32, print_cost = True):
    """
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 12288, number of training examples = 1080)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    X_test -- training set, of shape (input size = 12288, number of training examples = 120)
    Y_test -- test set, of shape (output size = 6, number of test examples = 120)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep consistent results
    seed = 3                                          # to keep consistent results
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of shape (n_x, n_y)
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_x, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
                ### END CODE HERE ###
                
                epoch_cost += minibatch_cost / num_minibatches

            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # lets save the parameters in a variable
        print(type(parameters))
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print ("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
        return parameters